<a href="https://colab.research.google.com/github/Maraudier/MatChatBot/blob/master/SQuADChatBotDataManage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Obtain your Kaggle token. 
token = {"username":"lightshift","key":"b7ab47deb22bd26d7cb071a39c232d1c"}
#Colab and Kaggle set up
from google.colab import files
!pip install kaggle
!mkdir .kaggle
!mkdir ~/.kaggle
import json 
with open('/content/.kaggle/kaggle.json', 'w') as file:
  json.dump(token, file)
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!kaggle datasets download -d stanfordu/stanford-question-answering-dataset -p /content
!unzip \*.zip
!rm -rf /content/dataset/dataset/
!ls

- path is now set to: {/content}
 57% 5.00M/8.73M [00:00<00:00, 15.4MB/s]
100% 8.73M/8.73M [00:00<00:00, 25.1MB/s]
Archive:  stanford-question-answering-dataset.zip
  inflating: dev-v1.1.json           
  inflating: train-v1.1.json         
dev-v1.1.json  stanford-question-answering-dataset.zip
sample_data    train-v1.1.json


In [0]:
import tensorflow as tf
import torch
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
  print('GPU confirmed: {}'.format(device_name))
else:
  raise SystemError("No GPU")

if torch.cuda.is_available():
  device = torch.device("cuda")
  print("%d GPU(s) " % torch.cuda.device_count())
  print("Using ", torch.cuda.get_device_name(0))

else:
  print('No GPU, cpu')

GPU confirmed: /device:GPU:0
1 GPU(s) 
Using  Tesla P100-PCIE-16GB


In [0]:
#Obtain our inputs
import numpy as np
import pandas as pd
import json
from subprocess import check_output

In [0]:
#data management step 1
squad_df = pd.read_json("train-v1.1.json", orient='columns')
#Only 2 columns, data includes nested json. More work to organize data 
squad_df.head()

,data,version
0,"{'title': 'University_of_Notre_Dame', 'paragra...",1.1
1,"{'title': 'Beyoncé', 'paragraphs': [{'context'...",1.1
2,"{'title': 'Montana', 'paragraphs': [{'context'...",1.1
3,"{'title': 'Genocide', 'paragraphs': [{'context...",1.1
4,"{'title': 'Antibiotics', 'paragraphs': [{'cont...",1.1


In [0]:
#Define a few functions to sort through the data further 
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'], verbose = 1):
  if verbose:
    print("Reading json...")
  file = json.loads(open(input_file_path).read())
  if verbose:
    print("Working...")
  js = pd.io.json.json_normalize(file , record_path )
  m = pd.io.json.json_normalize(file, record_path[:-1])
  r = pd.io.json.json_normalize(file,record_path[:-2])
  
  idx = np.repeat(r['context'].values, r.qas.str.len())
  ndx = np.repeat(m['id'].values, m['answers'].str.len())
  m['context'] = idx
  js['q_idx'] = ndx
  main =pd.concat([m[['id', 'question' , 'context']].set_index('id'),js.set_index('q_idx')], 1, sort=False).reset_index()
  main['c_id'] = main['context'].factorize()[0]
  if verbose:
    print("dataframe shape: {}".format(main.shape))
    print("Jobs Done")
  return main

def squad_json_to_dataframe_dev(input_file_path, record_path = ['data','paragraphs','qas','answers'], verbose = 1):
  if verbose:
    print("Reading json...")
  file = json.loads(open(input_file_path).read())
  if verbose:
    print("Working...")
  js = pd.io.json.json_normalize(file , record_path )
  m = pd.io.json.json_normalize(file, record_path[:-1] )
  r = pd.io.json.json_normalize(file,record_path[:-2])

  idx = np.repeat(r['contect'].values, r.qas.str.len())
  m['context'] = idx
  main = m[['id','question', 'context', 'answers']].set_index('id').reset_index()
  main['c_id'] = main['context'].factorize()[0]
  if verbose:
    print("Dataframe shape: {}".format(main.shape))
    print("Jobs Done")
  return main

In [0]:
input_file_path = 'train-v1.1.json'
record_path = ['data', 'paragraphs', 'qas', 'answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path)

Reading json...
Working...
dataframe shape: (87599, 6)
Jobs Done


In [0]:
train.head()

,index,question,context,answer_start,text,c_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",381,a Marian place of prayer and reflection,0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",92,a golden statue of the Virgin Mary,0


In [0]:
q_and_a = train[['question', 'text']].copy()
q_and_a.head()

,question,text
0,To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous
1,What is in front of the Notre Dame Main Building?,a copper statue of Christ
2,The Basilica of the Sacred heart at Notre Dame...,the Main Building
3,What is the Grotto at Notre Dame?,a Marian place of prayer and reflection
4,What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary
